In [114]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [121]:
df = pd.read_csv('LaunchData.csv')

In [122]:
df = df.drop(columns=['RocketName'])
x = df.drop(columns=['AltitudeHeight', 'TimeAtAltitude', 'TotalFlightTime'])
y = df[['AltitudeHeight', 'TimeAtAltitude', 'TotalFlightTime']]

In [123]:
x = pd.get_dummies(x, columns=['NoseType', 'FinShape'], drop_first=True)

In [124]:
xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size=0.2, random_state=42)

In [125]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(xTrain.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3)  
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [126]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

xTrain = xTrain.astype(np.float32)
xVal = xVal.astype(np.float32)
yTrain = yTrain.astype(np.float32)
yVal = yVal.astype(np.float32)

history = model.fit(
    xTrain, yTrain,
    validation_data=(xVal, yVal),
    epochs=200,
    batch_size=16,
    callbacks=[es],
    verbose=2
)

Epoch 1/200
12/12 - 1s - loss: 932986950166813605888.0000 - mae: 1685649408.0000 - val_loss: 60406781801920462848000.0000 - val_mae: 20985933824.0000 - 535ms/epoch - 45ms/step
Epoch 2/200
12/12 - 0s - loss: 932983642835837255680.0000 - mae: 1685730816.0000 - val_loss: 60406727758724934402048.0000 - val_mae: 20986023936.0000 - 42ms/epoch - 4ms/step
Epoch 3/200
12/12 - 0s - loss: 932979842923651661824.0000 - mae: 1685822208.0000 - val_loss: 60406682722728660697088.0000 - val_mae: 20986095616.0000 - 42ms/epoch - 3ms/step
Epoch 4/200
12/12 - 0s - loss: 932976324486442778624.0000 - mae: 1685941120.0000 - val_loss: 60406633183132759621632.0000 - val_mae: 20986146816.0000 - 45ms/epoch - 4ms/step
Epoch 5/200
12/12 - 0s - loss: 932972383836768829440.0000 - mae: 1685983872.0000 - val_loss: 60406574636337603805184.0000 - val_mae: 20986169344.0000 - 42ms/epoch - 3ms/step
Epoch 6/200
12/12 - 0s - loss: 932969920930722611200.0000 - mae: 1686038784.0000 - val_loss: 60406493571544311136256.0000 - val_

In [127]:
model.save('flightPathModel.h5')

import json
with open('columns.json', 'w') as f:
    json.dump(list(x.columns), f)